In [1]:
import pandas as pd
import glob
import re
import os
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import numpy as np
import string
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
from nltk.tokenize import word_tokenize

In [22]:
import warnings
from sklearn.feature_extraction.text import CountVectorizer
from datetime import datetime
from spellchecker import SpellChecker

#nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk import tokenize

pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
path = r'/Users/aliciafeuling/dsc-data-science-env/projects/Capstone-Project'
all_files = glob.glob(os.path.join(path, "*.csv"))
df_from_each_file = (pd.read_csv(f) for f in all_files)
blm_df = pd.concat(df_from_each_file, ignore_index=True)

In [4]:
blm_df.head()

,Unnamed: 0,date,geo,hashtags,nlikes,nreplies,nretweets,place,timezone,tweet,username
0,0,2020-06-24 16:59:09,NaN,['#blm'],20,NaN,NaN,NaN,NaN,"NO JUSTICE, NO PEACE #BLM pic.twitter.com/sFm...",satanic_syd
1,1,2020-06-24 16:52:19,NaN,"['#poem', '#motivation', '#blm']",206,NaN,NaN,NaN,NaN,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson
2,2,2020-06-24 16:47:56,NaN,['#blm'],23,NaN,NaN,NaN,NaN,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit
3,3,2020-06-24 16:46:02,NaN,['#blm'],147,NaN,NaN,NaN,NaN,My kid wrote #blm in chalk on my driveway and ...,Majhatesall
4,4,2020-06-24 16:43:54,NaN,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,NaN,NaN,NaN,NaN,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso


In [5]:
blm_df.shape

(227522, 11)

In [6]:
# Dropping the first column and the columns that aren't in every row that was created on the import
blm_df = blm_df.drop(["Unnamed: 0", 'geo', "nreplies", 'nretweets', 'place', 'timezone'], axis = 1)
blm_df.head()

,date,hashtags,nlikes,tweet,username
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM pic.twitter.com/sFm...",satanic_syd
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso


In [7]:
# Identifying missing values
blm_df.isna().sum()

date        1
hashtags    3
nlikes      3
tweet       1
username    1
dtype: int64

In [8]:
# dropping empty rows
blm_df = blm_df.dropna()
blm_df

,date,hashtags,nlikes,tweet,username
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM pic.twitter.com/sFm...",satanic_syd
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso
...,...,...,...,...,...
227517,2020-06-04 00:01:27,"['#blm', '#georgefloydmurder', '#policebrutali...",22,"Stay strong, everyone! \n\nWe WILL defend and...",Helenhs
227518,2020-06-04 00:01:07,"['#blacktranslivesmatter', '#blm']",30,"Before I go to bed, I just wanna say that in a...",terisasiagatonu
227519,2020-06-04 00:00:43,"['#superproudoftheirsolidarity', '#blm', '#res...",39,The Children Shall Lead... 🙌🏽 \n*Student orga...,akperera
227520,2020-06-04 00:00:27,"['#blacklivesmatter', '#blm', '#btsjin']",5420,The original plan was to donate to another cha...,haru130613_


In [9]:
blm_df.isna().sum()

date        0
hashtags    0
nlikes      0
tweet       0
username    0
dtype: int64

In [10]:
# Seeing how many rows were lost due to dropping NaNs
print('Dataset size:', blm_df.shape)
print('Columns are: ', blm_df.columns)

Dataset size: (227519, 5)
Columns are:  Index(['date', 'hashtags', 'nlikes', 'tweet', 'username'], dtype='object')


In [11]:
# Checking the datatypes
blm_df.dtypes

date        object
hashtags    object
nlikes      object
tweet       object
username    object
dtype: object

In [12]:
blm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 227519 entries, 0 to 227521
Data columns (total 5 columns):
date        227519 non-null object
hashtags    227519 non-null object
nlikes      227519 non-null object
tweet       227519 non-null object
username    227519 non-null object
dtypes: object(5)
memory usage: 10.4+ MB


In [13]:
def remove_picture_url(text):
    '''The input value is any text and the output removes the specific url used when a picture is added to a tweet.'''
    text = re.sub(r"pic.twitter.com\S+", '', text)
    return text

In [14]:
blm_df['tweet'] = blm_df['tweet'].apply(lambda x: remove_picture_url(x))
blm_df.head()

,date,hashtags,nlikes,tweet,username
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso


In [15]:
def remove_url(text):
    '''The input is any text and the output removes any url beginning with http.'''
    text = re.sub(r"http\S+","", text)
    return text

In [16]:
blm_df['tweet'] = blm_df['tweet'].apply(lambda x: remove_url(x))
blm_df.head()

,date,hashtags,nlikes,tweet,username
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso


In [17]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
# removing punctuation
def remove_punc(text):
    '''The input value is any text or column in a dataframe. The output is the text with punctuation and numbers removed.'''
    text = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', ' ', text)
    return text

In [19]:
blm_df['tweet_punct'] = blm_df['tweet'].apply(lambda x: remove_punc(x))
blm_df.head(10)

,date,hashtags,nlikes,tweet,username,tweet_punct
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd,NO JUSTICE NO PEACE BLM
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson,Funny thing\nAbout life is\nYou’ll always be t...
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall,My kid wrote blm in chalk on my driveway and I...
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso,Genuflect prostrate repent\n\nRepeat\n\n🤢🤢🤢\na...
5,2020-06-24 16:39:31,"['#blm', '#blacklivesmatter']",25,We are not done fighting folks.\n\nDon't let t...,49ersMike_NFL,We are not done fighting folks\n\nDont let the...
6,2020-06-24 16:36:00,['#blm'],26,#BLM\n\nArt by Kerry James Marshall,agitpopworld,BLM\n\nArt by Kerry James Marshall
7,2020-06-24 16:35:59,"['#historymatters', '#blm']",95,15. #HistoryMatters everyone needs to fight to...,almostjingo,HistoryMatters everyone needs to fight to pr...
8,2020-06-24 16:34:00,['#blm'],20,Some Americans feel that they're part of the l...,USC,Some Americans feel that theyre part of the la...
9,2020-06-24 16:32:57,['#blm'],98,Been seeing hella signs lately with people lev...,KeithHollandd,Been seeing hella signs lately with people lev...


In [26]:
# Tokenization
def tokenization(text):
    '''The input is text and the output is the text tokenized or separating the text into a list of words.'''
    text = re.split('\W+', text)
    return text


In [27]:
blm_df['tweet_tokenized'] = blm_df['tweet_punct'].apply(lambda x: tokenization(x.lower()))
blm_df.head()

,date,hashtags,nlikes,tweet,username,tweet_punct,tweet_tokenized
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd,NO JUSTICE NO PEACE BLM,"[no, justice, no, peace, blm, ]"
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson,Funny thing\nAbout life is\nYou’ll always be t...,"[funny, thing, about, life, is, you, ll, alway..."
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,"[protect, black, women, protect, black, men, p..."
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall,My kid wrote blm in chalk on my driveway and I...,"[my, kid, wrote, blm, in, chalk, on, my, drive..."
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso,Genuflect prostrate repent\n\nRepeat\n\n🤢🤢🤢\na...,"[genuflect, prostrate, repent, repeat, auspol,..."


In [28]:
# Defining stopwords using the NLTK Corpus
stopword = nltk.corpus.stopwords.words('english')

In [29]:
def remove_stopwords(text):
    '''Input is text and the output is that text with stopwords. Stopword list must first be defined.'''
    text = [word for word in text if word not in stopword]
    return text

In [30]:
blm_df['tweet_nonstop'] = blm_df['tweet_tokenized'].apply(lambda x: remove_stopwords(x))
blm_df.head()

,date,hashtags,nlikes,tweet,username,tweet_punct,tweet_tokenized,tweet_nonstop
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd,NO JUSTICE NO PEACE BLM,"[no, justice, no, peace, blm, ]","[justice, peace, blm, ]"
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson,Funny thing\nAbout life is\nYou’ll always be t...,"[funny, thing, about, life, is, you, ll, alway...","[funny, thing, life, always, one, else, prove,..."
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,"[protect, black, women, protect, black, men, p...","[protect, black, women, protect, black, men, p..."
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall,My kid wrote blm in chalk on my driveway and I...,"[my, kid, wrote, blm, in, chalk, on, my, drive...","[kid, wrote, blm, chalk, driveway, asked, neig..."
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso,Genuflect prostrate repent\n\nRepeat\n\n🤢🤢🤢\na...,"[genuflect, prostrate, repent, repeat, auspol,...","[genuflect, prostrate, repent, repeat, auspol,..."


In [31]:
# Stemming 
ps = nltk.PorterStemmer()

def stemming(text):
    '''Stemming takes the input of text and output is the stem of the words in the text. Stems are the roots of the words.'''
    text = [ps.stem(word) for word in text]
    return text

In [32]:
blm_df['tweet_stemmed'] = blm_df['tweet_nonstop'].apply(lambda x: stemming(x))
blm_df.head()

,date,hashtags,nlikes,tweet,username,tweet_punct,tweet_tokenized,tweet_nonstop,tweet_stemmed
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd,NO JUSTICE NO PEACE BLM,"[no, justice, no, peace, blm, ]","[justice, peace, blm, ]","[justic, peac, blm, ]"
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson,Funny thing\nAbout life is\nYou’ll always be t...,"[funny, thing, about, life, is, you, ll, alway...","[funny, thing, life, always, one, else, prove,...","[funni, thing, life, alway, one, els, prove, t..."
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,"[protect, black, women, protect, black, men, p...","[protect, black, women, protect, black, men, p...","[protect, black, women, protect, black, men, p..."
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall,My kid wrote blm in chalk on my driveway and I...,"[my, kid, wrote, blm, in, chalk, on, my, drive...","[kid, wrote, blm, chalk, driveway, asked, neig...","[kid, wrote, blm, chalk, driveway, ask, neighb..."
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso,Genuflect prostrate repent\n\nRepeat\n\n🤢🤢🤢\na...,"[genuflect, prostrate, repent, repeat, auspol,...","[genuflect, prostrate, repent, repeat, auspol,...","[genuflect, prostrat, repent, repeat, auspol, ..."


In [33]:
# Lemmatization
wn = nltk.WordNetLemmatizer()

def lemmatizer(text):
    '''Input is text and the output is the lemmatized text where similar words are grouped together.'''
    text = [wn.lemmatize(word) for word in text]
    return text

In [34]:
blm_df['tweet_lemmatized'] = blm_df['tweet_nonstop'].apply(lambda x: lemmatizer(x))
blm_df.head()

,date,hashtags,nlikes,tweet,username,tweet_punct,tweet_tokenized,tweet_nonstop,tweet_stemmed,tweet_lemmatized
0,2020-06-24 16:59:09,['#blm'],20,"NO JUSTICE, NO PEACE #BLM",satanic_syd,NO JUSTICE NO PEACE BLM,"[no, justice, no, peace, blm, ]","[justice, peace, blm, ]","[justic, peac, blm, ]","[justice, peace, blm, ]"
1,2020-06-24 16:52:19,"['#poem', '#motivation', '#blm']",206,Funny thing\nAbout life is\nYou’ll always be t...,derickijohnson,Funny thing\nAbout life is\nYou’ll always be t...,"[funny, thing, about, life, is, you, ll, alway...","[funny, thing, life, always, one, else, prove,...","[funni, thing, life, alway, one, els, prove, t...","[funny, thing, life, always, one, else, prove,..."
2,2020-06-24 16:47:56,['#blm'],23,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,AngelofLightLit,PROTECT BLACK WOMEN\nPROTECT BLACK MEN\nPROTEC...,"[protect, black, women, protect, black, men, p...","[protect, black, women, protect, black, men, p...","[protect, black, women, protect, black, men, p...","[protect, black, woman, protect, black, men, p..."
3,2020-06-24 16:46:02,['#blm'],147,My kid wrote #blm in chalk on my driveway and ...,Majhatesall,My kid wrote blm in chalk on my driveway and I...,"[my, kid, wrote, blm, in, chalk, on, my, drive...","[kid, wrote, blm, chalk, driveway, asked, neig...","[kid, wrote, blm, chalk, driveway, ask, neighb...","[kid, wrote, blm, chalk, driveway, asked, neig..."
4,2020-06-24 16:43:54,"['#auspol', '#blm', '#racismeantiblancs', '#vi...",24,"Genuflect, prostrate, repent.\n\nRepeat.\n\n🤢🤢...",CaroDiRusso,Genuflect prostrate repent\n\nRepeat\n\n🤢🤢🤢\na...,"[genuflect, prostrate, repent, repeat, auspol,...","[genuflect, prostrate, repent, repeat, auspol,...","[genuflect, prostrat, repent, repeat, auspol, ...","[genuflect, prostrate, repent, repeat, auspol,..."


In [35]:
blm_df.dtypes

date                object
hashtags            object
nlikes              object
tweet               object
username            object
tweet_punct         object
tweet_tokenized     object
tweet_nonstop       object
tweet_stemmed       object
tweet_lemmatized    object
dtype: object

In [36]:
# Converting the date column to datetime
blm_df['date'] = pd.to_datetime(blm_df['date'], infer_datetime_format=True, errors='coerce')
blm_df.dtypes

date                datetime64[ns]
hashtags                    object
nlikes                      object
tweet                       object
username                    object
tweet_punct                 object
tweet_tokenized             object
tweet_nonstop               object
tweet_stemmed               object
tweet_lemmatized            object
dtype: object

In [37]:
blm_df['nlikes'] = pd.to_numeric(blm_df['nlikes'], errors='coerce')
blm_df.dtypes

date                datetime64[ns]
hashtags                    object
nlikes                     float64
tweet                       object
username                    object
tweet_punct                 object
tweet_tokenized             object
tweet_nonstop               object
tweet_stemmed               object
tweet_lemmatized            object
dtype: object

In [38]:
# Exporting the clean tweets dataframe to a csv to be used for the EDA notebook
blm_df.to_csv('Cleaned_BLM_Tweets.csv', index=True)

In [31]:
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove punctuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)
    text = [ps.stem(word) for word in tokens if word not in stopword] # remove stopwords and stemming
    return text

In [32]:
# Vectorization
countVectorizer = CountVectorizer(analyzer=clean_text)
countVector = countVectorizer.fit_transform(blm_df['tweet'])
print('{} Number of tweets has {} words'.format(countVector.shape[0], countVector.shape[1]))

227519 Number of tweets has 195546 words


In [33]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names())
count_vect_df.head()

,,a,aa,aaa,aaaa,aaaaa,aaaaaaaahahaha,aaaaaaannnnd,aaaaaannnndddd,aaaaahmericaaaa,...,𝙞𝙣,𝙥𝙚𝙖𝙘𝙛𝙪𝙡𝙡𝙮,𝙩𝙝𝙚,𝙬𝙤𝙧𝙙,𝙾𝙿𝙿𝚁𝙴𝚂𝚂𝙸𝙾𝙽,𝟎𝟔𝟐𝟒,𝟏𝟎,𝟏𝟑𝐭𝐡,𝟙𝟝,𝟣𝟤
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
count_vect_df.to_csv('Count_vector_BLM_tweets.csv')